**KFold ile Model Skoru**

In [93]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression

In [94]:
train_data= pd.read_csv("house_train.csv")

In [95]:
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [1]:
#eksik satırları sil, yani label sütunu boş olan satırları sil
#subset hangi sütunda eksik veri kontrolü yapılacağını söylüyor.
#dropna "na"=eksik veri
train_data.dropna(axis=0, subset=["SalePrice"], inplace=True)

NameError: name 'train_data' is not defined

In [97]:
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [98]:
#Datada NaN yazan kısımlar null değerlerdir, şimdi bu sütunları sileceğiz.

train_data.drop(['LotFrontage', 'GarageYrBlt', 'MasVnrArea', 'Alley', 'PoolQC', 'MiscFeature'], axis=1, inplace=True)

In [99]:
train_data.head()

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,3SsnPorch,ScreenPorch,PoolArea,Fence,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,0,0,0,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,NaN,0,12,2008,WD,Normal,250000


In [100]:
# sonuç değişkenini al
y=train_data.SalePrice

# sonuç değişkenini çıkar
train_data.drop(["SalePrice"], axis=1, inplace=True)

In [101]:
#kategorikleri atıp sadece numeriklere bakıyoruz
"""cname= columns name, burada veri setinin sütunları üzerinde döngü yapılır
ve cname her bir sütunu temsil eder. Eğer veri tipleri belirtildiği gibiyse 
bu listeye eklenir."""
numeric_cols= [cname for cname in train_data.columns if train_data[cname].dtype in ["int64", "float64"]]

X= train_data[numeric_cols].copy()
X

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,1,60,8450,7,5,2003,2003,706,0,150,...,548,0,61,0,0,0,0,0,2,2008
1,2,20,9600,6,8,1976,1976,978,0,284,...,460,298,0,0,0,0,0,0,5,2007
2,3,60,11250,7,5,2001,2002,486,0,434,...,608,0,42,0,0,0,0,0,9,2008
3,4,70,9550,7,5,1915,1970,216,0,540,...,642,0,35,272,0,0,0,0,2,2006
4,5,60,14260,8,5,2000,2000,655,0,490,...,836,192,84,0,0,0,0,0,12,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,7917,6,5,1999,2000,0,0,953,...,460,0,40,0,0,0,0,0,8,2007
1456,1457,20,13175,6,6,1978,1988,790,163,589,...,500,349,0,0,0,0,0,0,2,2010
1457,1458,70,9042,7,9,1941,2006,275,0,877,...,252,0,60,0,0,0,0,2500,5,2010
1458,1459,20,9717,5,6,1950,1996,49,1029,0,...,240,366,0,112,0,0,0,0,4,2010


In [102]:
print("Input datanın şekli: {} ve sonuç değişkenin şekli: {}".format(X.shape,y.shape))

Input datanın şekli: (1460, 34) ve sonuç değişkenin şekli: (1460,)


Burada modeli görebilmek adına veri ön işleme kısmını hızlıca geçtik.

In [103]:
#K= n_splits= 5i shuffle=karıştır
kf= KFold(n_splits= 5, shuffle=True, random_state=42)
kf

KFold(n_splits=5, random_state=42, shuffle=True)

In [104]:
cnt=1 #kaçıncı kat olduğu, count,sayac vb.

# split() methodu Train-Test olarak ayırmak için bize indeksleri döner
for train_index, test_index in kf.split(X,y):
    print(f'Fold:{cnt}, Train set: {len(train_index)}, Test set: {len(test_index)}')
    cnt+=1

Fold:1, Train set: 1168, Test set: 292
Fold:2, Train set: 1168, Test set: 292
Fold:3, Train set: 1168, Test set: 292
Fold:4, Train set: 1168, Test set: 292
Fold:5, Train set: 1168, Test set: 292


In [105]:
# RMSE'yi hesaplamak için - (eksi) ile çarpacağız çünkü  cross_val_score() dan bize negatif gelecek.

def rmse(score):
    rmse= np.sqrt(-score)
    print(f'rmse = {"{:.2f}".format(rmse)}')

In [106]:
# Lineer Regresyon modeli

# score için verdiğimiz değer MSE'nin negatifi, çünkü MSE aslında maliyet demek
# yüksek MSE istemediğimiz için negatif olarak verdik

score= cross_val_score(LinearRegression(), X, y, cv=kf, scoring= "neg_mean_squared_error")

In [107]:
print(f'Her bir fold için score: {score}')

rmse(score.mean())

Her bir fold için score: [-1.39334669e+09 -1.32533433e+09 -3.39493937e+09 -9.31045536e+08
 -7.16620849e+08]
rmse = 39398.70


In [108]:
#mean kullnamadan rmse hesaplama
np.sqrt(-sum(score) /len(score))

39398.697377565666

**K-Fold Cross Validation - Classification Örneği**

In [115]:
train_data = pd.read_csv('titanic_train.csv')

In [117]:
train_data.dropna(axis=0, subset=['Survived'], inplace=True)

In [118]:
y = train_data.Survived

In [119]:
train_data.drop(['Survived'], axis=1, inplace=True)

In [120]:
train_data.drop(['Age'], axis=1, inplace=True)

In [121]:
train_data.head()

,PassengerId,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,0,0,373450,8.0500,NaN,S


In [122]:
# sadece numerik kolonları seç

numeric_cols = [cname for cname in train_data.columns if train_data[cname].dtype in ['int64', 'float64']]

In [123]:
X = train_data[numeric_cols].copy()

In [125]:
X.head()

,PassengerId,Pclass,SibSp,Parch,Fare
0,1,3,1,0,7.2500
1,2,1,1,0,71.2833
2,3,3,0,0,7.9250
3,4,1,1,0,53.1000
4,5,3,0,0,8.0500


In [124]:
print("Train datanın şekli: {} ve sonuç değişkenin şekli: {}".format(X.shape, y.shape))

Train datanın şekli: (891, 5) ve sonuç değişkenin şekli: (891,)


In [126]:
# ilk 5 train datasını göster, X ve y yi bşrleştirerek dtanın tamamını gösterir 

pd.concat([X,y], axis=1).head()

,PassengerId,Pclass,SibSp,Parch,Fare,Survived
0,1,3,1,0,7.2500,0
1,2,1,1,0,71.2833,1
2,3,3,0,0,7.9250,1
3,4,1,1,0,53.1000,1
4,5,3,0,0,8.0500,0


**StratifiedKFold ile Model**

In [127]:
# K = 5
# Stratified K-Fold yaptığımız için foldlardaki sınıf oranları gerçek dataya ile yakın olacaktır.
kf= StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
kf

StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

In [128]:
cnt=1

for train_index, test_index in kf.split(X,y):
    print(f'Fold: {cnt}, Train set: {len(train_index)}, Test set: {len(test_index)}')
    cnt+=1

Fold: 1, Train set: 712, Test set: 179
Fold: 2, Train set: 713, Test set: 178
Fold: 3, Train set: 713, Test set: 178
Fold: 4, Train set: 713, Test set: 178
Fold: 5, Train set: 713, Test set: 178


**Not:** cross_val_score()'da cv parametresini boş geçerseniz default olarak StratifiedKFold yapacaktır.



**Logistic Regression Modeli**

In [129]:
# CV skorunu alalım
# skorlama yöntemi -> accuracy


score = cross_val_score(LogisticRegression(random_state= 42), X, y, cv = kf, scoring="accuracy")

In [132]:
print(f'Her bir fold scoru: {score}')
print(f'Ortalama Skor: {"{:.2f}".format(score.mean())}')

Her bir fold scoru: [0.66480447 0.69662921 0.70224719 0.69101124 0.66292135]
Ortalama Skor: 0.68


**Hyperparameter Tuning**

In [134]:
"""Burada belli parametreleri kullanarak döngüye hepsini vereceğiz ve
hangisin en iyi sonucu verdiğini göreceğiz."""
# Logistic Regression'ın bütün Solver'larını deneylim
solvers= ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']

# her bir solver için ortalama score hesaplayalım
# max_iter'i 4000 verdik
for solver in solvers:
    score= cross_val_score(LogisticRegression(max_iter=4000, solver=solver, random_state=42),
                          X,y, cv=kf, scoring="accuracy")
    print(f'Ortalama Skor({solver}): {"{:.3f}".format(score.mean())}')

Ortalama Skor(newton-cg): 0.684
Ortalama Skor(lbfgs): 0.684
Ortalama Skor(liblinear): 0.684
Ortalama Skor(sag): 0.678
Ortalama Skor(saga): 0.681
